# Queries with and without Azure OpenAI

So far, you have your Search Engine loaded **from two different data sources in two diferent text-based indexes**, on this notebook we are going to try some example queries and then use Azure OpenAI service to see if we can get even better results.

The idea is that a user can ask a question about Computer Science (first datasource/index) or about Covid (second datasource/index), and the engine will respond accordingly.
This **Multi-Index** demo, mimics the scenario where a company loads multiple type of documents of different types and about completly different topics and the search engine must respond with the most relevant results.

## Install Dependencies


In [9]:
%pip install -r common/requirements.txt


[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


## Set up variables

In [10]:
import os
import urllib
import requests
import random
import json
from collections import OrderedDict
from IPython.display import display, HTML, Markdown
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.chat_models import AzureChatOpenAI
from langchain.docstore.document import Document
from langchain.chains.qa_with_sources import load_qa_with_sources_chain
from langchain.embeddings import AzureOpenAIEmbeddings

from common.prompts import COMBINE_QUESTION_PROMPT, COMBINE_PROMPT, COMBINE_PROMPT_TEMPLATE
from common.utils import (
    get_search_results,
    model_tokens_limit,
    num_tokens_from_docs,
    num_tokens_from_string
)

from dotenv import load_dotenv
load_dotenv("credentials.env", override=True)


True

In [11]:
# Setup the Payloads header
headers = {'Content-Type': 'application/json','api-key': os.environ['AZURE_SEARCH_KEY']}
params = {'api-version': os.environ['AZURE_SEARCH_API_VERSION']}

## Multi-Index Search queries

In [12]:
# Text-based Indexes that we are going to query (from Notebook 01 and 02)
index1_name = "cogsrch-index-files"
index2_name = "cogsrch-index-csv"
indexes = [index1_name, index2_name]

Try questions that you think might be answered or addressed in computer science papers in 2020-2021 or that can be addressed by medical publications about COVID in 2020-2021. Try comparing the results with the open version of ChatGPT.<br>
The idea is that the answers using Azure OpenAI only looks at the information contained on these publications.

**Example Questions you can ask**:
- What is CLP?
- How Markov chains work?
- What are some examples of reinforcement learning?
- What are the main risk factors for Covid-19?
- What medicine reduces inflamation in the lungs?
- Why Covid doesn't affect kids that much compared to adults?
- Does chloroquine really works against covid?
- Who won the 1994 soccer world cup? # This question should yield no answer if the system is correctly grounded

In [13]:
QUESTION = "What is CLP?"

### Search on both indexes individually and aggragate results

#### **Note**: 
In order to standarize the indexes, **there must be 8 mandatory fields present on each text-based index**: `id, title, content, chunks, language, name, location, vectorized`. This is so that each document can be treated the same along the code. Also, **all indexes must have a semantic configuration**.

In [14]:
agg_search_results = dict()

for index in indexes:
    search_payload = {
        "search": QUESTION,
        "select": "id, title, chunks, name, location",
        "queryType": "semantic",
        "semanticConfiguration": "my-semantic-config",
        "count": "true",
        "speller": "lexicon",
        "queryLanguage": "en-us",
        "captions": "extractive",
        "answers": "extractive",
        "top": "10"
    }

    r = requests.post(os.environ['AZURE_SEARCH_ENDPOINT'] + "/indexes/" + index + "/docs/search",
                     data=json.dumps(search_payload), headers=headers, params=params)
    print(r.status_code)

    search_results = r.json()
    agg_search_results[index]=search_results
    print("Index:", index, "Results Found: {}, Results Returned: {}".format(search_results['@odata.count'], len(search_results['value'])))

200
Index: cogsrch-index-files Results Found: 9789, Results Returned: 10
200
Index: cogsrch-index-csv Results Found: 48638, Results Returned: 10


In [15]:
r.text

'{"@odata.context":"https://gptkb-3hhfqf2n5qzbc.search.windows.net/indexes(\'cogsrch-index-csv\')/$metadata#docs(*)","@odata.count":48638,"@search.answers":[{"key":"u2g30x1j","text":"Isotype-specific antibody responses to rotavirus and virus proteins in cows inoculated with subunit vaccines composed of recombinant SA11 rotavirus core-like particles (CLP) or virus-like particles (VLP).","highlights":"Isotype-specific antibody responses to rotavirus and virus proteins in cows inoculated with subunit vaccines composed of <em>recombinant SA11 rotavirus core-like particles</em> (CLP) or virus-like particles (VLP).","score":0.9345703125}],"value":[{"@search.score":12.707714,"@search.rerankerScore":2.791743278503418,"@search.captions":[{"text":"Immunosorbent electron microscopy was used to quantify recombinant baculovirus-generated bluetongue virus (BTV) core-like particles (CLP) in either purified preparations or lysates of recombinant baculovirus-infected cells. The capture antibody was an 

### Display the top results (from both searches) based on the score

In [16]:
display(HTML('<h4>Top Answers</h4>'))

for index,search_results in agg_search_results.items():
    for result in search_results['@search.answers']:
        if result['score'] > 0.5: # Show answers that are at least 50% of the max possible score=1
            display(HTML('<h5>' + 'Answer - score: ' + str(round(result['score'],2)) + '</h5>'))
            display(HTML(result['text']))
            
print("\n\n")
display(HTML('<h4>Top Results</h4>'))

content = dict()
ordered_content = OrderedDict()


for index,search_results in agg_search_results.items():
    for result in search_results['value']:
        if result['@search.rerankerScore'] > 1:# Show answers that are at least 25% of the max possible score=4
            content[result['id']]={
                                    "title": result['title'],
                                    "chunks": result['chunks'], 
                                    "chunks_vectors": [],
                                    "name": result['name'], 
                                    "location": result['location'] ,
                                    "caption": result['@search.captions'][0]['text'],
                                    "score": result['@search.rerankerScore'],
                                    "index": index
                                    }
    
#After results have been filtered we will Sort and add them as an Ordered list\n",
for id in sorted(content, key= lambda x: content[x]["score"], reverse=True):
    ordered_content[id] = content[id]
    url = str(ordered_content[id]['location']) + os.environ['BLOB_SAS_TOKEN']
    title = str(ordered_content[id]['title']) if (ordered_content[id]['title']) else ordered_content[id]['name']
    score = str(round(ordered_content[id]['score'],2))
    display(HTML('<h5><a href="'+ url + '">' + title + '</a> - score: '+ score + '</h5>'))
    display(HTML(ordered_content[id]['caption']))

## Comments on Query results

As seen above the semantic search feature of Azure Cognitive Search service is good. It gives answers (sometimes) and also the top results with the corresponding file and the paragraph where the answers is possible located.

Let's see if we can make this better with Azure OpenAI

# Using Azure OpenAI

To use OpenAI to get a better answer to our question, the thought process is simple: let's **give the answer and the content of the documents from the search result to the GPT model as context and let it provide a better response**.

Now, before we do this, we need to understand a few things first:

1) Chainning and Prompt Engineering
2) Embeddings

We will use a library call **LangChain** that wraps a lot of boiler plate code.
Langchain is one library that does a lot of the prompt engineering for us under the hood, for more information see [here](https://python.langchain.com/en/latest/index.html)

In [17]:
# Set the ENV variables that Langchain needs to connect to Azure OpenAI
os.environ["OPENAI_API_VERSION"] = os.environ["AZURE_OPENAI_API_VERSION"]

**Important Note**: Starting now, we will utilize OpenAI models. Please ensure that you have deployed the following models within the Azure OpenAI portal using these precise deployment names:

- text-embedding-ada-002
- gpt-35-turbo
- gpt-35-turbo-16k
- gpt-4
- gpt-4-32k

Should you have deployed the models under different names, the code provided below will not function as expected. To resolve this, you would need to modify the variable names throughout all the notebooks.

## A gentle intro to chaining LLMs and prompt engineering

Chains are what you get by connecting one or more large language models (LLMs) in a logical way. (Chains can be built of entities other than LLMs but for now, let’s stick with this definition for simplicity).

Azure OpenAI is a type of LLM (provider) that you can use but there are others like Cohere, Huggingface, etc.

Chains can be simple (i.e. Generic) or specialized (i.e. Utility).

* Generic — A single LLM is the simplest chain. It takes an input prompt and the name of the LLM and then uses the LLM for text generation (i.e. output for the prompt).

Here’s an example:

In [18]:
MODEL = "gpt-4-32k" # options: gpt-35-turbo, gpt-35-turbo-16k, gpt-4, gpt-4-32k
COMPLETION_TOKENS = 1000
llm = AzureChatOpenAI(deployment_name=MODEL, temperature=0, max_tokens=COMPLETION_TOKENS)

In [19]:
# Now we create a simple prompt template
prompt = PromptTemplate(
    input_variables=["question", "language"],
    template='Answer the following question: "{question}". Give your response in {language}',
)

print(prompt.format(question=QUESTION, language="French"))

Answer the following question: "What is CLP?". Give your response in French


In [20]:
# And finnaly we create our first generic chain
chain_chat = LLMChain(llm=llm, prompt=prompt)
chain_chat({"question": QUESTION, "language": "French"})

{'question': 'What is CLP?',
 'language': 'French',
 'text': 'CLP est l\'acronyme de "Classification, Labelling and Packaging", qui se traduit en français par "Classification, Étiquetage et Emballage". Il s\'agit d\'un règlement de l\'Union Européenne concernant la classification, l\'étiquetage et l\'emballage de substances et de mélanges chimiques.'}

**Note**: this is the first time you use OpenAI in this Accelerator, so if you get a Resource not found error, is most likely because the name of your OpenAI model deployment is different than the variable MODEL set above

Great!!, now you know how to create a simple prompt and use a chain in order to answer a general question using ChatGPT knowledge!. 

It is important to note that we rarely use generic chains as standalone chains. More often they are used as building blocks for Utility chains (as we will see next). Also important to notice is that we are NOT using our documents or the result of the Azure Search yet, just the knowledge of ChatGPT on the data it was trained on.

**The second type of Chains are Utility:**

* Utility — These are specialized chains, comprised of many LLMs to help solve a specific task. For example, LangChain supports some end-to-end chains (such as QA_WITH_SOURCES for QnA Doc retrieval, Summarization, etc) and some specific ones (such as GraphQnAChain for creating, querying, and saving graphs). 

We will look at one specific chain called **qa_with_sources** in this workshop for digging deeper and solve our use case of enhancing the results of Azure Cognitive Search.


But before dealing with the utility chain needed, we need to deal first with this problem: **the content of the search result files is or can be very lengthy, more than the allowed tokens allowed by the GPT Azure OpenAI models**. 

This is where the concept of embeddings/vectors come into place.

## Embeddings and Vector Search

From the Azure OpenAI documentation ([HERE](https://learn.microsoft.com/en-us/azure/ai-services/openai/how-to/embeddings?tabs=python)), An embedding is a special format of data representation that can be easily utilized by machine learning models and algorithms. The embedding is an information dense representation of the semantic meaning of a piece of text. Each embedding is a vector of floating point numbers, such that the distance between two embeddings in the vector space is correlated with semantic similarity between two inputs in the original format. For example, if two texts are similar, then their vector representations should also be similar. 

To address the challenge of accommodating context within the token limit of a Language Model (LLM), the solution involves the following steps:

1. **Segmenting Documents**: Divide the documents into smaller segments or chunks.
2. **Vectorization of Chunks**: Transform these chunks into vectors using appropriate techniques.
3. **Vector Semantic Search**: Execute a semantic search using vectors to identify the top chunks similar to the given question.
4. **Optimal Context Provision**: Provide the LLM with the most relevant and concise context, thereby achieving an optimal balance between comprehensiveness and lengthiness.


Notice that **the documents chunks are already done in Azure Search**. *ordered_content* dictionary (created a few cells above) contains the chunks of each document. So we don't really need to chunk them again, but we still need to make sure that we can be as fast as possible and that we are below the max allowed input token limits of our selected OpenAI model.

Our ultimate goal is to rely solely on vector indexes. While it is possible to manually code parsers with OCR for various file types and develop a scheduler to synchronize data with the index, there is a more efficient alternative: **Azure Cognitive Search is soon going to release automated chunking strategies and vectorization within the next months**, so we have three options: 
1. Wait for this functionality while in the meantime manually push chunks and its vectors to the vector-based indexes 
2. Fill up the vector-based indexes on-demand, as documents are discovered by users
3. Use custom skills (for chunking and vectorization) and use knowledge stores in order to create a vector-base index from a text-based-ai-enriched index at ingestion time. See [HERE](https://github.com/Azure/cognitive-search-vector-pr/blob/main/demo-python/code/azure-search-vector-ingestion-python-sample.ipynb) for instructions on how to do this.

In this notebook we are going to implement Option 2: **Create vector-based indexes per each text-based indexes and fill them up on-demand as documents are discovered**. Why? because is simpler and quick to implement, while we wait for Option 1 to become a feature of Azure Search Engine (which is the automation of Option 3 inside the search engine).

As observed in Notebooks 1 and 2, each text-based index contains a field named `vectorized` that we have not utilized yet. We will now harness this field. The objective is to avoid vectorizing all documents at the time of ingestion (Option 3). Instead, we can vectorize the chunks as users search for or discover documents. This approach ensures that we allocate funds and resources only when the documents are actually required. Typically, in an organization with a vast repository of documents in a data lake, only 20% of the documents are frequently accessed, while the rest remain untouched. This phenomenon mirrors the [Pareto Principle](https://en.wikipedia.org/wiki/Pareto_principle) found in nature.

In [21]:
index_name = "cogsrch-index-files"
index2_name = "cogsrch-index-csv"
indexes = [index_name, index2_name]

In order to not duplicate code, we have put many of the code used above into functions. These functions are in the `common/utils.py` and `common/prompts.py` files. This way we can use these functios in the app that we will build later.

In [22]:
k = 10 # Number of results per each text_index
ordered_results = get_search_results(QUESTION, indexes, k=10, reranker_threshold=1)
print("Number of results:",len(ordered_results))

{'@odata.context': "https://gptkb-3hhfqf2n5qzbc.search.windows.net/indexes('cogsrch-index-files')/$metadata#docs(*)", '@odata.count': 9789, '@search.answers': [{'key': 'aHR0cHM6Ly9kYXRhc2V0c2dwdHNtYXJ0c2VhcmNoLmJsb2IuY29yZS53aW5kb3dzLm5ldC9hcnhpdmNzL3BkZi8wNzAxLzA3MDEwODJ2MS5wZGY1', 'text': 'The key notions of CLP are those of an algebra and an associated constraint solver over a class of constraints, namely a set of first order formulas including the always satisfiable constraint true, the un- satisfiable constraint false, and closed under variable renaming, conjunction and existential quantification.', 'highlights': 'The key notions of CLP are<em> those of an algebra and an associated constraint solver over a class of constraints,</em> namely a set of first order formulas including the always satisfiable constraint true, the un- satisfiable constraint false, and closed under variable renaming, conjunction and existential quantification.', 'score': 0.96875}], 'value': [{'@search.score

In [23]:
# Uncomment the below line if you want to inspect the ordered results
# ordered_results

Now we can fill up the vector-based index as users lookup documents using the text-based index. This approach although it requires two searches per user query (one on the text-based indexes and the other one on the vector-based indexes), it is simpler to implement and will be incrementatly faster as user use the system.

In [24]:
embedder = AzureOpenAIEmbeddings(model="text-embedding-ada-002", skip_empty=True) 

In [26]:
%%time
for key,value in ordered_results.items():
    if value["vectorized"] != True: # If the document has not been vectorized yet
        i = 0
        print("Vectorizing",len(value["chunks"]),"chunks from Document:",value["location"])
        for chunk in value["chunks"]: # Iterate over the document's text chunks
            try:
                upload_payload = {  # Insert the chunk and its vector in the vector-based index
                    "value": [
                        {
                            "id": key + "_" + str(i),
                            "title": f"{value['title']}_chunk_{str(i)}",
                            "chunk": chunk,
                            "chunkVector": embedder.embed_query(chunk if chunk!="" else "-------"),
                            "name": value["name"],
                            "location": value["location"],
                            "@search.action": "upload"
                        },
                    ]
                }

                r = requests.post(os.environ['AZURE_SEARCH_ENDPOINT'] + "/indexes/" + value["index"]+"-vector" + "/docs/index",
                                     data=json.dumps(upload_payload), headers=headers, params=params)
                
                if r.status_code != 200:
                    print(r.status_code)
                    print(r.text)
                else:
                    i = i + 1 # increment chunk number
                
            except Exception as e:
                print("Exception:",e)
                print(chunk)
                continue

        # Update document in text-based index and mark it as "vectorized"
        upload_payload = {
            "value": [
                {
                    "id": key,
                    "vectorized": True,
                    "@search.action": "merge"
                },
            ]
        }

        r = requests.post(os.environ['AZURE_SEARCH_ENDPOINT'] + "/indexes/" + value["index"]+ "/docs/index",
                                     data=json.dumps(upload_payload), headers=headers, params=params)

Vectorizing 7 chunks from Document: https://datasetsgptsmartsearch.blob.core.windows.net/arxivcs/pdf/0508/0508108v1.pdf
Vectorizing 5 chunks from Document: https://datasetsgptsmartsearch.blob.core.windows.net/arxivcs/pdf/0701/0701082v1.pdf
Vectorizing 8 chunks from Document: https://datasetsgptsmartsearch.blob.core.windows.net/arxivcs/pdf/0508/0508106v1.pdf
Vectorizing 8 chunks from Document: https://datasetsgptsmartsearch.blob.core.windows.net/arxivcs/pdf/0011/0011030v1.pdf
Vectorizing 14 chunks from Document: https://datasetsgptsmartsearch.blob.core.windows.net/arxivcs/pdf/0506/0506005v1.pdf
Vectorizing 13 chunks from Document: https://datasetsgptsmartsearch.blob.core.windows.net/arxivcs/pdf/0408/0408056v1.pdf
Vectorizing 11 chunks from Document: https://datasetsgptsmartsearch.blob.core.windows.net/arxivcs/pdf/0310/0310042v1.pdf
Vectorizing 8 chunks from Document: https://datasetsgptsmartsearch.blob.core.windows.net/arxivcs/pdf/0003/0003026v1.pdf
Vectorizing 8 chunks from Document: h

**Note**: How the text-based and the vector-based indexes stay in sync?
For document changes, the problem is already taken care of, since Azure Engine will update the text-based index automatically if a file has a new version. This puts the vectorized field in None and the next time that the file is searched it will be vectorized again into the vector-based index.

However for deletion of files, the problem is half solved. Azure Search engine would delete the documents in the text-based index if the file is deleted on the source, however you will need to code a script that runs on a fixed schedule that looks for deleted ids in the text-based index and deletes the corresponding chunks in the vector-based index.

Now we search on the vector-based indexes and get the top k most similar chunks to our question:

In [27]:
vector_indexes = [index+"-vector" for index in indexes]

k = 10
similarity_k = 3
ordered_results = get_search_results(QUESTION, vector_indexes,
                                        k=k, # Number of results per vector index
                                        reranker_threshold=1,
                                        vector_search=True, 
                                        similarity_k=similarity_k,
                                        query_vector = embedder.embed_query(QUESTION)
                                        )
print("Number of results:",len(ordered_results))

{'@odata.context': "https://gptkb-3hhfqf2n5qzbc.search.windows.net/indexes('cogsrch-index-files-vector')/$metadata#docs(*)", '@odata.count': 98, '@search.answers': [{'key': 'aHR0cHM6Ly9kYXRhc2V0c2dwdHNtYXJ0c2VhcmNoLmJsb2IuY29yZS53aW5kb3dzLm5ldC9hcnhpdmNzL3BkZi8wMzEwLzAzMTAwNDJ2MS5wZGY1_2', 'text': 'A CLP(FD) program searches a solution for a set of variables which take values over finite domains and which must verify a set of constraints. The evolution of the domains can be viewed as a sequence of applications of reduction operators attached to the constraints.', 'highlights': 'A CLP(FD) program searches<em> a solution for a set of variables which take values over finite domains and which must verify a set of constraints.</em> The evolution of the domains can be viewed as a sequence of applications of reduction operators attached to the constraints.', 'score': 0.9228515625}], 'value': [{'@search.score': 0.010989011265337467, '@search.rerankerScore': 3.0801165103912354, '@search.caption

For vector search is not recommended to give more than k=5 chunks (of max 5000 characters each) to the LLM as context. Otherwise you can have issues later with the token limit trying to have a conversation with memory.

In [28]:
top_docs = []
for key,value in ordered_results.items():
    location = value["location"] if value["location"] is not None else ""
    top_docs.append(Document(page_content=value["chunk"], metadata={"source": location}))
        
print("Number of chunks:",len(top_docs))

Number of chunks: 3


In [29]:
# Calculate number of tokens of our docs
if(len(top_docs)>0):
    tokens_limit = model_tokens_limit(MODEL) # this is a custom function we created in common/utils.py
    prompt_tokens = num_tokens_from_string(COMBINE_PROMPT_TEMPLATE) # this is a custom function we created in common/utils.py
    context_tokens = num_tokens_from_docs(top_docs) # this is a custom function we created in common/utils.py
    
    requested_tokens = prompt_tokens + context_tokens + COMPLETION_TOKENS
    
    chain_type = "map_reduce" if requested_tokens > 0.9 * tokens_limit else "stuff"  
    
    print("System prompt token count:",prompt_tokens)
    print("Max Completion Token count:", COMPLETION_TOKENS)
    print("Combined docs (context) token count:",context_tokens)
    print("--------")
    print("Requested token count:",requested_tokens)
    print("Token limit for", MODEL, ":", tokens_limit)
    print("Chain Type selected:", chain_type)
        
else:
    print("NO RESULTS FROM AZURE SEARCH")

System prompt token count: 1669
Max Completion Token count: 1000
Combined docs (context) token count: 945
--------
Requested token count: 3614
Token limit for gpt-4-32k : 32768
Chain Type selected: stuff


Now we will use our Utility Chain from LangChain `qa_with_sources`

In [30]:
if chain_type == "stuff":
    chain = load_qa_with_sources_chain(llm, chain_type=chain_type, 
                                       prompt=COMBINE_PROMPT)
elif chain_type == "map_reduce":
    chain = load_qa_with_sources_chain(llm, chain_type=chain_type, 
                                       question_prompt=COMBINE_QUESTION_PROMPT,
                                       combine_prompt=COMBINE_PROMPT,
                                       return_intermediate_steps=True)

In [31]:
%%time
# Try with other language as well
response = chain({"input_documents": top_docs, "question": QUESTION, "language": "English"})

CPU times: total: 0 ns
Wall time: 17.7 s


In [32]:
display(Markdown(response['output_text']))

CLP can refer to Consultation-Liaison Psychiatry, a field of psychiatry that involves providing psychiatric services in outpatient medical clinics<sup><a href="https://api.elsevier.com/content/article/pii/S0033318220301420; https://www.sciencedirect.com/science/article/pii/S0033318220301420?v=s5" target="_blank">[1]</a></sup>. It can also refer to Cecal Ligation and Puncture, a method used in medical research, particularly in studies related to sepsis<sup><a href="https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5809586/" target="_blank">[2]</a></sup>. Additionally, in the context of virology, CLP stands for Core-Like Particles, which are assembled in vitro in the study of alphaviruses<sup><a href="https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7103146/" target="_blank">[3]</a></sup>. The specific meaning of CLP would depend on the context in which it is used.

**Please Note**: There are some instances where, despite the answer's high accuracy and quality, the references are not done according to the instructions provided in the COMBINE_PROMPT. This behavior is anticipated when dealing with GPT-3.5 models. We will provide a more detailed explanation of this phenomenon towards the conclusion of Notebook 5.

In [ ]:
# Uncomment if you want to inspect the results from map_reduce chain type, each top similar chunk summary (k=4 by default)

# if chain_type == "map_reduce":
#     for step in response['intermediate_steps']:
#         display(HTML("<b>Chunk Summary:</b> " + step))

# Summary
##### This answer is way better than taking just the result from Azure Cognitive Search. So the summary is:
- Utilizing Azure Cognitive Search, we conduct a multi-index text-based search that identifies the top documents from each index.
- Utilizing Azure Cognitive Search's vector search, we extract the most relevant chunks of information.
- Subsequently, Azure OpenAI utilizes these extracted chunks as context, comprehends the content, and employs it to deliver optimal answers.
- Best of two worlds!

# NEXT
In the next notebook, we are going to see how we can treat complex and large documents separately, also using Vector Search